<a href="https://colab.research.google.com/github/Priya170205/composio/blob/master/original_template/Sesame_CSM_(1B)-TTS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### News

Placeholder

### Installation

In [6]:
# Install yt-dlp for downloading YouTube audio
# Install pydub for audio conversion
!pip install yt-dlp pydub soundfile



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.0/176.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 44.6 MB/s eta 0:00:00


In [7]:
# Replace with the YouTube video URL
youtube_url = "https://www.youtube.com/watch?v=FHIml6cRz0k&ab_channel=SandeepMaheshwari"


In [ ]:
import os

# Download audio as MP3
!yt-dlp -x --audio-format mp3 {youtube_url}

# List all mp3 files downloaded
mp3_files = [f for f in os.listdir() if f.endswith(".mp3")]
print("Downloaded MP3 files:", mp3_files)



[youtube] Extracting URL: https://www.youtube.com/watch?v=FHIml6cRz0k
[youtube] FHIml6cRz0k: Downloading webpage
[youtube] FHIml6cRz0k: Downloading tv simply player API JSON
[youtube] FHIml6cRz0k: Downloading tv client config
[youtube] FHIml6cRz0k: Downloading player 6742b2b9-main
[youtube] FHIml6cRz0k: Downloading tv player API JSON
[info] FHIml6cRz0k: Downloading 1 format(s): 251
[download] Destination: 99.99% Don’t Know The Self ｜ By Sandeep Maheshwari ｜ Motivational Video Hindi [FHIml6cRz0k].webm
[download] 100% of   19.76MiB in 00:00:00 at 23.34MiB/s
[ExtractAudio] Destination: 99.99% Don’t Know The Self ｜ By Sandeep Maheshwari ｜ Motivational Video Hindi [FHIml6cRz0k].mp3


In [ ]:
# Take the first downloaded mp3 file
original_mp3 = mp3_files[0]
new_mp3 = "audio.mp3"

os.rename(original_mp3, new_mp3)
print(f"Renamed '{original_mp3}' to '{new_mp3}'")



In [ ]:
from pydub import AudioSegment

# Convert to WAV
audio = AudioSegment.from_mp3(new_mp3)
audio.export("audio.wav", format="wav")
print("Converted to audio.wav")


In [ ]:
from pydub import AudioSegment, effects

audio = AudioSegment.from_file("audio.wav")
normalized_audio = effects.normalize(audio)
normalized_audio.export("audio_normalized.wav", format="wav")


In [ ]:
from pydub import AudioSegment

# Load your 30-min audio
audio = AudioSegment.from_file("audio.wav")
os.makedirs("chunks", exist_ok=True)
# Split into 30-second chunks (or 10-second if you want more samples)
chunk_length_ms = 10 * 1000  # 30 seconds
chunks = []
overlap_ms = 2000         # 2-second overlap

for start_ms in range(0, len(audio), chunk_length_ms - overlap_ms):
    chunk = audio[start_ms:start_ms + chunk_length_ms]

for i, start_ms in enumerate(range(0, len(audio), chunk_length_ms)):
    chunk = audio[start_ms:start_ms + chunk_length_ms]
    chunk_name = f"chunks/voice_chunk_{i}.wav"
    chunk.export(chunk_name, format="wav")
    chunks.append(chunk_name)

print("Chunks created:", chunks[:5], "...")  # preview first 5 chunks


In [ ]:
print("Total chunks:", len(chunks))


In [ ]:
!pip install faster-whisper

from faster_whisper import WhisperModel
import os # Import the os module

model = WhisperModel("large", device="cuda", compute_type="float16")  # use "cuda" if GPU
chunk_folder = "chunks"
transcripts = []



In [ ]:
import json
chunk_folder = "chunks"

# List all .wav files
chunk_files = sorted([f for f in os.listdir(chunk_folder) if f.endswith(".wav")])

# Transcription loop with progress
transcripts = []
for idx, file in enumerate(chunk_files):
    path = os.path.join(chunk_folder, file)
    print(f"Starting transcription of chunk {idx+1}/{len(chunk_files)}: {file} ...")
    result = model.transcribe(path)
    text = result.text
    segments = result.segments

    # Optional: print word-level timestamps for this chunk
    for seg in segments:
        print("Segment:", seg.text)
        for w in seg.words:
            print(f"Word: {w.word}, start: {w.start:.2f}s, end: {w.end:.2f}s")


    # Save results
    transcripts.append({
        "audio": path,
        "text": text,
        "segments": [
            {
                "segment_text": seg.text,
                "words": [{"word": w.word, "start": w.start, "end": w.end} for w in seg.words]
            } for seg in segments
        ]
    })

    print(f"Chunk {idx+1} done ✅")

# Save JSON and JSONL
with open("voice_dataset.json", "w") as f:
    json.dump(transcripts, f, indent=2)

with open("voice_dataset.jsonl", "w") as f:
    for entry in transcripts:
        f.write(json.dumps(entry) + "\n")


In [ ]:
##test
import json

# Load the JSON dataset
with open("voice_dataset.json", "r") as f:
    transcripts = json.load(f)

print(f"Total chunks: {len(transcripts)}")


In [ ]:
from IPython.display import Audio, display

# Choose a few random chunks to check
for i in [1,4,3]:  # first 3 chunks
    print(f"--- Chunk {i} ---")
    print("Transcript:", transcripts[i]["text"])
    display(Audio(transcripts[i]["audio"]))


In [ ]:
import soundfile as sf

audio_data, sampling_rate = sf.read("audio.wav")
print("Audio length (samples):", len(audio_data))
print("Sampling rate:", sampling_rate)


In [ ]:
from IPython.display import Audio

Audio("audio.wav")


In [ ]:
# Placeholder

### Unsloth

`FastModel` supports loading nearly any model now! This includes Vision and Text models!

In [ ]:
# Install unsloth
!pip install unsloth[colab-new] torch accelerate peft transformers bitsandbytes trl xformers --upgrade

In [ ]:
from unsloth import FastModel
from transformers import CsmForConditionalGeneration
import torch

model, processor = FastModel.from_pretrained(
    model_name = "unsloth/csm-1b",
    max_seq_length= 2048, # Choose any for long context!
    dtype = None, # Leave as None for auto-detection
    auto_model = CsmForConditionalGeneration,
    load_in_4bit = False, # Select True for 4bit - reduces memory usage
)

In [ ]:
from unsloth import FastModel
from transformers import CsmForConditionalGeneration
import torch

model, processor = FastModel.from_pretrained(
    model_name = "unsloth/csm-1b",
    max_seq_length= 2048, # Choose any for long context!
    dtype = None, # Leave as None for auto-detection
    auto_model = CsmForConditionalGeneration,
    load_in_4bit = False, # Select True for 4bit - reduces memory usage
)

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [ ]:
model = FastModel.get_peft_model(
    model,
    r = 32, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

<a name="Data"></a>
### Data Prep  

We will use the `MrDragonFox/Elise`, which is designed for training TTS models. Ensure that your dataset follows the required format: **text, audio** for single-speaker models or **source, text, audio** for multi-speaker models. You can modify this section to accommodate your own dataset, but maintaining the correct structure is essential for optimal training.

In [ ]:
#@title Dataset Prep functions
from datasets import load_dataset, Audio, Dataset
import os
from transformers import AutoProcessor
processor = AutoProcessor.from_pretrained("unsloth/csm-1b")

raw_ds = load_dataset("MrDragonFox/Elise", split="train")

# Getting the speaker id is important for multi-speaker models and speaker consistency
speaker_key = "source"
if "source" not in raw_ds.column_names and "speaker_id" not in raw_ds.column_names:
    print("Unsloth: No speaker found, adding default \"source\" of 0 for all examples")
    new_column = ["0"] * len(raw_ds)
    raw_ds = raw_ds.add_column("source", new_column)
elif "source" not in raw_ds.column_names and "speaker_id" in raw_ds.column_names:
    speaker_key = "speaker_id"

target_sampling_rate = 24000
raw_ds = raw_ds.cast_column("audio", Audio(sampling_rate=target_sampling_rate))

def preprocess_example(example):
    conversation = [
        {
            "role": str(example[speaker_key]),
            "content": [
                {"type": "text", "text": example["text"]},
                {"type": "audio", "path": example["audio"]["array"]},
            ],
        }
    ]

    try:
        model_inputs = processor.apply_chat_template(
            conversation,
            tokenize=True,
            return_dict=True,
            output_labels=True,
            text_kwargs = {
                "padding": "max_length", # pad to the max_length
                "max_length": 256, # this should be the max length of audio
                "pad_to_multiple_of": 8,
                "padding_side": "right",
            },
            audio_kwargs = {
                "sampling_rate": 24_000,
                "max_length": 240001, # max input_values length of the whole dataset
                "padding": "max_length",
            },
            common_kwargs = {"return_tensors": "pt"},
        )
    except Exception as e:
        print(f"Error processing example with text '{example['text'][:50]}...': {e}")
        return None

    required_keys = ["input_ids", "attention_mask", "labels", "input_values", "input_values_cutoffs"]
    processed_example = {}
    # print(model_inputs.keys())
    for key in required_keys:
        if key not in model_inputs:
            print(f"Warning: Required key '{key}' not found in processor output for example.")
            return None

        value = model_inputs[key][0]
        processed_example[key] = value


    # Final check (optional but good)
    if not all(isinstance(processed_example[key], torch.Tensor) for key in processed_example):
         print(f"Error: Not all required keys are tensors in final processed example. Keys: {list(processed_example.keys())}")
         return None

    return processed_example

processed_ds = raw_ds.map(
    preprocess_example,
    remove_columns=raw_ds.column_names,
    desc="Preprocessing dataset",
)

<a name="Train"></a>
### Train the model
Now let's use Huggingface  `Trainer`! More docs here: [Transformers docs](https://huggingface.co/docs/transformers/main_classes/trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`.

In [ ]:
from transformers import TrainingArguments, Trainer
from unsloth import is_bfloat16_supported

trainer = Trainer(
    model = model,
    train_dataset = processed_ds,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01, # Turn this on if overfitting
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

In [ ]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

In [ ]:
trainer_stats = trainer.train()

In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

<a name="Inference"></a>
### Inference
Let's run the model! You can change the prompts

In [ ]:
from IPython.display import Audio, display
import soundfile as sf

text = "We just finished fine tuning a text to speech model... and it's pretty good!"
speaker_id = 0
inputs = processor(f"[{speaker_id}]{text}", add_special_tokens=True).to("cuda")
audio_values = model.generate(
    **inputs,
    max_new_tokens=125, # 125 tokens is 10 seconds of audio, for longer speech increase this
    # play with these parameters to tweak results
    # depth_decoder_top_k=0,
    # depth_decoder_top_p=0.9,
    # depth_decoder_do_sample=True,
    # depth_decoder_temperature=0.9,
    # top_k=0,
    # top_p=1.0,
    # temperature=0.9,
    # do_sample=True,
    #########################################################
    output_audio=True
)
audio = audio_values[0].to(torch.float32).cpu().numpy()
sf.write("example_without_context.wav", audio, 24000)
display(Audio(audio, rate=24000))

In [ ]:
text = "Sesame is a super cool TTS model which can be fine tuned with Unsloth."

speaker_id = 0
# Another equivalent way to prepare the inputs
conversation = [
    {"role": str(speaker_id), "content": [{"type": "text", "text": text}]},
]
audio_values = model.generate(
    **processor.apply_chat_template(
        conversation,
        tokenize=True,
        return_dict=True,
    ).to("cuda"),
    max_new_tokens=125, # 125 tokens is 10 seconds of audio, for longer speech increase this
    # play with these parameters to tweak results
    # depth_decoder_top_k=0,
    # depth_decoder_top_p=0.9,
    # depth_decoder_do_sample=True,
    # depth_decoder_temperature=0.9,
    # top_k=0,
    # top_p=1.0,
    # temperature=0.9,
    # do_sample=True,
    #########################################################
    output_audio=True
)
audio = audio_values[0].to(torch.float32).cpu().numpy()
sf.write("example_without_context.wav", audio, 24000)
display(Audio(audio, rate=24000))

#### Voice and style consistency

Sesame CSM's power comes from providing audio context for each speaker. Let's pass a sample utterance from our dataset to ground speaker identity and style.

In [ ]:
speaker_id = 0

utterance = raw_ds[3]["audio"]["array"]
utterance_text = raw_ds[3]["text"]
text = "Sesame is a super cool TTS model which can be fine tuned with Unsloth."

# CSM will fill in the audio for the last text.
# You can even provide a conversation history back in as you generate new audio

conversation = [
    {"role": str(speaker_id), "content": [{"type": "text", "text": utterance_text},{"type": "audio", "path": utterance}]},
    {"role": str(speaker_id), "content": [{"type": "text", "text": text}]},
]

inputs = processor.apply_chat_template(
        conversation,
        tokenize=True,
        return_dict=True,
    )
audio_values = model.generate(
    **inputs.to("cuda"),
    max_new_tokens=125, # 125 tokens is 10 seconds of audio, for longer text increase this
    # play with these parameters to tweak results
    # depth_decoder_top_k=0,
    # depth_decoder_top_p=0.9,
    # depth_decoder_do_sample=True,
    # depth_decoder_temperature=0.9,
    # top_k=0,
    # top_p=1.0,
    # temperature=0.9,
    # do_sample=True,
    #########################################################
    output_audio=True
)
audio = audio_values[0].to(torch.float32).cpu().numpy()
sf.write("example_with_context.wav", audio, 24000)
display(Audio(audio, rate=24000))

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.save_pretrained("lora_model")  # Local saving
processor.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# processor.push_to_hub("your_name/lora_model", token = "...") # Online saving

### Saving to float16

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", processor, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", processor, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", processor, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", processor, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False:
    model.save_pretrained("model")
    processor.save_pretrained("model")
if False:
    model.push_to_hub("hf/model", token = "")
    processor.push_to_hub("hf/model", token = "")
